- repr(), bytes() 등 객체를 다른 방식으로 표현하는 내장 함수의 지원
- 클래스 메서드로 대안 생성자 구현
- format() 내장 함수와 str.format() 메서드에서 사용하는 포맷 언어 확장
- 읽기 전용 접근만 허용하는 속성 제공
- 집합 및 딕셔너리 키로 사용할 수 있도록 객체를 해시 가능하게 만들기
- __slots__를 이용해서 메모리 절약하기

### 9.1 객체 표현
- repr() : 객체를 개발자가 보고자 하는 형태로 표현한 문자열로 반환한다.
- str() : 객체를 사용자가 보고자 하는 형태로 표현한 문자열로 반환한다.

### 9.2 벡터 클래스의 부활

In [16]:
from array import array
import math

class Vector2d:
    typecode = 'd'
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
        
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + 
               bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    

In [17]:
v1 = Vector2d(3,4)
print (v1.x, v1.y)
x,y = v1
print (x, y)
print (v1)

Vector2d(3.0, 4.0)
v1_clone = eval(repr(v1))
print (v1 == v1_clone)
print (v1)
octets = bytes(v1)
print (octets)
print (abs(v1))
print (bool(v1), bool(Vector2d(0, 0)))

3.0 4.0
3.0 4.0
(3.0, 4.0)
True
(3.0, 4.0)
b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'
5.0
True False


### 9.3 대안 생성자

In [1]:
@classmethod
def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:].cast(typecode))
    return cls(*memv)

### 9.4 @classmethod와 @staticmethod

In [4]:
class Demo:
    @classmethod
    def klassmeth(*args):
        return args
    @staticmethod
    def statmeth(*args):
        return args

print (Demo.klassmeth())
print (Demo.klassmeth('spam'))
print (Demo.)

(<class '__main__.Demo'>,)
(<class '__main__.Demo'>, 'spam')
